# Make a simple Flask site that streams video from the camera

In [15]:
from flask import Flask, render_template, Response

app = Flask(__name__)

import cv2
import numpy as np
import pyrealsense2 as rs

def gen():
    frame_width = 640
    frame_height = 480

    pipeline = rs.pipeline()
    config = rs.config()
    config.enable_stream(rs.stream.color, frame_width, frame_height, rs.format.bgr8, 30)
    config.enable_stream(rs.stream.depth, frame_width, frame_height, rs.format.z16, 30)

    
    # Create an align object
    align_to = rs.stream.color
    align = rs.align(align_to)

    pipeline.start(config)
    try:
        while True:
            frames = pipeline.wait_for_frames()
            # Align the depth frame to color frame
            aligned_frames = align.process(frames)

            # Get aligned frames
            depth_frame = aligned_frames.get_depth_frame() # aligned_depth_frame is a 640x480 depth image
            color_frame = aligned_frames.get_color_frame()
            if not depth_frame or not color_frame:
                continue
                
            # Convert images to numpy arrays
            depth_image = np.asanyarray(depth_frame.get_data())
            color_image = np.asanyarray(color_frame.get_data())
            
            # Apply colormap on depth image (image must be converted to 8-bit per pixel first)
            depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)

            # Stack both images horizontally
            images = np.hstack((color_image, depth_colormap))
            
            #Encode and deploy
            ret, jpeg = cv2.imencode('.jpg', images)
            frame = jpeg.tobytes()
            yield (b'--frame\r\n'
                   b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n\r\n')
            
    finally:

        # Stop streaming
        pipeline.stop()

@app.route('/')
def video_feed():
    return Response(gen(),
                    mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == '__main__':
    app.run(host='0.0.0.0', debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [10/Jan/2020 17:21:17] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Jan/2020 17:22:27] "GET / HTTP/1.1" 500 -
Error on request:
Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/werkzeug/serving.py", line 304, in run_wsgi
    execute(self.server.app)
  File "/usr/local/lib/python3.5/dist-packages/werkzeug/serving.py", line 294, in execute
    for data in application_iter:
  File "/usr/local/lib/python3.5/dist-packages/werkzeug/wsgi.py", line 506, in __next__
    return self._next()
  File "/usr/local/lib/python3.5/dist-packages/werkzeug/wrappers/base_response.py", line 45, in _iter_encoded
    for item in iterable:
  File "<ipython-input-15-5bc5ebad9ee8>", line 23, in gen
    
RuntimeError: xioctl(VIDIOC_S_FMT) failed Last Error: Device or resource busy
127.0.0.1 - - [10/Jan/2020 17:22:27] "GET /favicon.ico HTTP/1.1" 404 -
192.168.1.235 - - [10/Jan/2020 17:29:53] "GET / HTTP/1.1" 500 -
E